In [1]:
import numpy as np
import pandas as pd

# Instalando Spark

In [9]:
pip install sparkmagic

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: numpy in c:\programdata\anaconda3\lib\site-packages (from sparkmagic) (1.20.1)



ERROR: Could not install packages due to an OSError: [WinError 5] Acesso negado: 'c:\\programdata\\anaconda3\\lib\\site-packages\\__pycache__\\nest_asyncio.cpython-38.pyc'
Consider using the `--user` option or check the permissions.



  Created wheel for sparkmagic: filename=sparkmagic-0.20.0-py3-none-any.whl size=66525 sha256=e471807cc88ad635ae232b3b29403b98db6034da778c3f31493394315c4d219c
  Stored in directory: c:\users\notebook\appdata\local\pip\cache\wheels\42\58\b0\b5e033a82e461f67965b293fb2f665593fd1a7f6b107ad6444
  Created wheel for autovizwidget: filename=autovizwidget-0.20.0-py3-none-any.whl size=14641 sha256=e72e6af2995db300fd9b7f860236c61df9f1c5186bdc3a04080efd7440407c13
  Stored in directory: c:\users\notebook\appdata\local\pip\cache\wheels\c0\c6\72\ade57ec305cbd10071a63cb4e2508e14e4cf3b5340c6cfa801
  Created wheel for hdijupyterutils: filename=hdijupyterutils-0.20.0-py3-none-any.whl size=7661 sha256=20b4a4ed7b1b2e0c63259db0b2cff042cb3e89bbf8b3f3ec0d480f528eec3184
  Stored in directory: c:\users\notebook\appdata\local\pip\cache\wheels\00\75\a5\d7d33027c25fbf73c0104a501d9003280241ba1c18b5147dac
Successfully built sparkmagic autovizwidget hdijupyterutils
  Attempting uninstall: nest-asyncio
    Found exist

In [10]:
pip install pyspark

# Importando bibliotecas Spark SQL e Spark ML

In [35]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

spark = SparkSession.builder.master("local[*]").getOrCreate()

# Carregando o dado

In [18]:
csv = spark.read.csv('C:/Users/Notebook/Documents/Notebooks/dados/ml_with_spark/flights.csv', inferSchema=True, header=True)
csv.show(10)

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
|        19|        5|     DL|          10397|        15016|      -1|     -19|
|        19|        5|     DL|          15016|        10397|       0|      -1|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|      

# Preparando o dado para o modelo de Classificação

In [20]:
data = csv.select("DayofMonth", "DayofWeek", "Carrier", "OriginAirportID", "DestAirportID", "DepDelay",
                  ((col("ArrDelay") > 15).cast("Int").alias("label")))
data.show(18)

+----------+---------+-------+---------------+-------------+--------+-----+
|DayofMonth|DayofWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|label|
+----------+---------+-------+---------------+-------------+--------+-----+
|        19|        5|     DL|          11433|        13303|      -3|    0|
|        19|        5|     DL|          14869|        12478|       0|    0|
|        19|        5|     DL|          14057|        14869|      -4|    0|
|        19|        5|     DL|          15016|        11433|      28|    1|
|        19|        5|     DL|          11193|        12892|      -6|    0|
|        19|        5|     DL|          10397|        15016|      -1|    0|
|        19|        5|     DL|          15016|        10397|       0|    0|
|        19|        5|     DL|          10397|        14869|      15|    1|
|        19|        5|     DL|          10397|        10423|      33|    1|
|        19|        5|     DL|          11278|        10397|     323|    1|
|        19|


# Dividindo o dado

In [22]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print("Training Rows.", train_rows, "Testing Rows:", test_rows)

Training Rows. 1891562 Testing Rows: 810656


# Define the Pipeline

In [38]:
strIdx = StringIndexer(inputCol = "Carrier", outputCol = "CarrierIdx")
catVect = VectorAssembler(inputCols = ["CarrierIdx", "DayofMonth", "DayofWeek", "OriginAirportID", "DestAirportID"],
                          outputCol="catFeatures")
catIdx = VectorIndexer(inputCol = catVect.getOutputCol(), outputCol = "idxCatFeatures")
numVect = VectorAssembler(inputCols = ["DepDelay"], outputCol = "normfeatures")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normFeatures")
featVect = VectorAssembler(inputCols = ["idxCatFeatures", "normFeatures"], outputCol = "features")
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)
pipeline = Pipeline(stages=[strIdx, catVect, catIdx, numVect, minMax, featVect, lr])

# Executando o pipeline para treino do modelo

In [39]:
pipLineModel = pipeline.fit(train)

In [40]:
prediction = pipLineModel.transform(test)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show(100, truncate=False)

+---------------------------------------------------+----------+---------+
|features                                           |prediction|trueLabel|
+---------------------------------------------------+----------+---------+
|[10.0,1.0,0.0,10397.0,12191.0,0.03115264797507788] |0.0       |0        |
|[10.0,1.0,0.0,10397.0,12264.0,0.04205607476635514] |0.0       |0        |
|[10.0,1.0,0.0,10397.0,13851.0,0.03167185877466251] |0.0       |0        |
|[10.0,1.0,0.0,10423.0,13487.0,0.027518172377985463]|0.0       |0        |
|[10.0,1.0,0.0,10423.0,14869.0,0.05036344755970924] |0.0       |1        |
|[10.0,1.0,0.0,10529.0,11193.0,0.037383177570093455]|0.0       |0        |
|[10.0,1.0,0.0,10529.0,11433.0,0.030114226375908618]|0.0       |0        |
|[10.0,1.0,0.0,10693.0,10397.0,0.030114226375908618]|0.0       |0        |
|[10.0,1.0,0.0,10693.0,11433.0,0.029595015576323987]|0.0       |0        |
|[10.0,1.0,0.0,10693.0,13487.0,0.030633437175493248]|0.0       |0        |
|[10.0,1.0,0.0,10721.0,12

# Compute Confusion Matrix

In [41]:
tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())
pr = tp / (tp + fp)
re = tp / (tp + fn)
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", pr),
 ("Recall", re),
 ("F1", 2*pr*re/(re+pr))],["metric", "value"])
metrics.show()

Py4JJavaError: An error occurred while calling o521.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 40.0 failed 1 times, most recent failure: Lost task 0.0 in stage 40.0 (TID 92) (DESKTOP-TGUU6OU executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark\spark-3.3.0-bin-hadoop3\python\lib\pyspark.zip\pyspark\worker.py", line 540, in main
RuntimeError: Python in worker has different version 3.9 than that in driver 3.8, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:559)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:765)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:747)
	at org.apache.spark.api.python.BasePythonRunner$

# Review the Area Under ROC

In [42]:
evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
aur = evaluator.evaluate(prediction)
print ("AUR = ", aur)

AUR =  0.9227695818380741
